In [1]:
import sys
sys.path

['',
 '/usr/lib/python2.7/site-packages/avro-1.7.7-py2.7.egg',
 '/usr/lib64/python27.zip',
 '/usr/lib64/python2.7',
 '/usr/lib64/python2.7/plat-linux2',
 '/usr/lib64/python2.7/lib-tk',
 '/usr/lib64/python2.7/lib-old',
 '/usr/lib64/python2.7/lib-dynload',
 '/home/kranthidr/.local/lib/python2.7/site-packages',
 '/usr/lib64/python2.7/site-packages',
 '/usr/lib64/python2.7/site-packages/gtk-2.0',
 '/usr/lib/python2.7/site-packages',
 '/usr/lib/python2.7/site-packages/IPython/extensions',
 '/home/kranthidr/.ipython']

In [2]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
findspark.find()

'/usr/hdp/current/spark2-client'

In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("ch28Recommendations").getOrCreate()

In [4]:
sc = spark.sparkContext
sc._jsc.sc().uiWebUrl().get()

u'http://gw02.itversity.com:4042'

In [5]:
for x in sc._conf.getAll():
    if 'PROXY' in x[0]:
        print(x[1])
print(spark.conf.get('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES'))
print(spark.conf.get('spark.driver.appUIAddress'))

rm01.itversity.com
http://rm01.itversity.com:19288/proxy/application_1533622723243_11682
http://rm01.itversity.com:19288/proxy/application_1533622723243_11682
http://gw02.itversity.com:4042


In [6]:
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [7]:
# ratings = spark.read.text("/data/sample_movielens_ratings.txt")\
#   .rdd.toDF()\                           #WHY this Step ??
#   .selectExpr("split(value , '::') as col")\
#   .selectExpr(
#     "cast(col[0] as int) as userId",
#     "cast(col[1] as int) as movieId",
#     "cast(col[2] as float) as rating",
#     "cast(col[3] as long) as timestamp")

In [8]:
ratings1 = spark.read.text("/user/kranthidr/dataSets/spark-guide/data/sample_movielens_ratings.txt")

In [9]:
type(ratings1)

pyspark.sql.dataframe.DataFrame

In [10]:
ratings1.show()

+--------------------+
|               value|
+--------------------+
| 0::2::3::1424380312|
| 0::3::1::1424380312|
| 0::5::2::1424380312|
| 0::9::4::1424380312|
|0::11::1::1424380312|
|0::12::2::1424380312|
|0::15::1::1424380312|
|0::17::1::1424380312|
|0::19::1::1424380312|
|0::21::1::1424380312|
|0::23::1::1424380312|
|0::26::3::1424380312|
|0::27::1::1424380312|
|0::28::1::1424380312|
|0::29::1::1424380312|
|0::30::1::1424380312|
|0::31::1::1424380312|
|0::34::1::1424380312|
|0::37::1::1424380312|
|0::41::2::1424380312|
+--------------------+
only showing top 20 rows



In [11]:
ratings = ratings1.selectExpr("split(value , '::') as col")\
  .selectExpr(
    "cast(col[0] as int) as userId",
    "cast(col[1] as int) as movieId",
    "cast(col[2] as float) as rating",
    "cast(col[3] as long) as timestamp")

In [12]:
ratings.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     0|      2|   3.0|1424380312|
|     0|      3|   1.0|1424380312|
|     0|      5|   2.0|1424380312|
|     0|      9|   4.0|1424380312|
|     0|     11|   1.0|1424380312|
|     0|     12|   2.0|1424380312|
|     0|     15|   1.0|1424380312|
|     0|     17|   1.0|1424380312|
|     0|     19|   1.0|1424380312|
|     0|     21|   1.0|1424380312|
|     0|     23|   1.0|1424380312|
|     0|     26|   3.0|1424380312|
|     0|     27|   1.0|1424380312|
|     0|     28|   1.0|1424380312|
|     0|     29|   1.0|1424380312|
|     0|     30|   1.0|1424380312|
|     0|     31|   1.0|1424380312|
|     0|     34|   1.0|1424380312|
|     0|     37|   1.0|1424380312|
|     0|     41|   2.0|1424380312|
+------+-------+------+----------+
only showing top 20 rows



In [13]:
training, test = ratings.randomSplit([1.8, 0.2])

In [14]:
# Number of Alternating Iteration to solve A = XY equation

als = ALS(rank=10, 
          maxIter=10, 
          regParam=0.01, 
          numUserBlocks=10, 
          numItemBlocks=10, 
          implicitPrefs=False, 
          alpha=1.0, 
          userCol='userId', 
          itemCol='movieId', 
          seed=None, 
          ratingCol='rating', 
          nonnegative=False, 
          checkpointInterval=10, 
          intermediateStorageLevel='MEMORY_AND_DISK', 
          finalStorageLevel='MEMORY_AND_DISK', 
          coldStartStrategy='drop')

# als = ALS().setsetMaxIter(5).setRegParam(0.01).setUserCol("userId").setItemCol("movieId").setRatingCol("rating")

In [15]:
print(als.explainParams())

alpha: alpha for implicit preference (default: 1.0, current: 1.0)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10, current: 10)
coldStartStrategy: strategy for dealing with unknown or new users/items at prediction time. This may be useful in cross-validation or production scenarios, for handling user/item ids the model has not seen in the training data. Supported values: 'nan', 'drop'. (default: nan, current: drop)
finalStorageLevel: StorageLevel for ALS model factors. (default: MEMORY_AND_DISK, current: MEMORY_AND_DISK)
implicitPrefs: whether to use implicit preference (default: False, current: False)
intermediateStorageLevel: StorageLevel for intermediate datasets. Cannot be 'NONE'. (default: MEMORY_AND_DISK, current: MEMORY_AND_DISK)
itemCol: column name for item ids. Ids mus

In [16]:
alsModel = als.fit(training)
predictions = alsModel.transform(test)

In [17]:
# COMMAND ----------
user_recommend = alsModel.recommendForAllUsers(10)

In [18]:
user_recommend.take(2)

[Row(userId=28, recommendations=[Row(movieId=92, rating=4.895199775695801), Row(movieId=81, rating=4.8838276863098145), Row(movieId=52, rating=4.865847110748291), Row(movieId=12, rating=4.815539360046387), Row(movieId=71, rating=4.422400951385498), Row(movieId=8, rating=4.38348388671875), Row(movieId=2, rating=4.175174713134766), Row(movieId=89, rating=4.03819465637207), Row(movieId=49, rating=4.008114814758301), Row(movieId=80, rating=3.9377405643463135)]),
 Row(userId=26, recommendations=[Row(movieId=92, rating=6.227603435516357), Row(movieId=8, rating=6.058201789855957), Row(movieId=88, rating=5.094727516174316), Row(movieId=22, rating=5.054958343505859), Row(movieId=23, rating=4.988158226013184), Row(movieId=24, rating=4.905362129211426), Row(movieId=7, rating=4.775722026824951), Row(movieId=98, rating=4.297576427459717), Row(movieId=71, rating=4.1704206466674805), Row(movieId=68, rating=4.167436599731445)])]

In [19]:
user_recommend.printSchema()

root
 |-- userId: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- movieId: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [20]:
user_recommend.show(1)

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    28|[[92, 4.8952], [8...|
+------+--------------------+
only showing top 1 row



In [21]:
user_recommend.selectExpr("userId", "explode(recommendations)").show()    

+------+---------------+
|userId|            col|
+------+---------------+
|    28|   [92, 4.8952]|
|    28|[81, 4.8838277]|
|    28| [52, 4.865847]|
|    28|[12, 4.8155394]|
|    28| [71, 4.422401]|
|    28|  [8, 4.383484]|
|    28| [2, 4.1751747]|
|    28|[89, 4.0381947]|
|    28| [49, 4.008115]|
|    28|[80, 3.9377406]|
|    26|[92, 6.2276034]|
|    26|  [8, 6.058202]|
|    26|[88, 5.0947275]|
|    26|[22, 5.0549583]|
|    26| [23, 4.988158]|
|    26| [24, 4.905362]|
|    26|  [7, 4.775722]|
|    26|[98, 4.2975764]|
|    26|[71, 4.1704206]|
|    26|[68, 4.1674366]|
+------+---------------+
only showing top 20 rows



In [22]:
item_recommend  = alsModel.recommendForAllItems(10)

In [23]:
item_recommend.printSchema()

root
 |-- movieId: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- userId: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [24]:
item_recommend.first()

Row(movieId=31, recommendations=[Row(userId=8, rating=3.0425338745117188), Row(userId=14, rating=2.973376512527466), Row(userId=6, rating=2.953794002532959), Row(userId=7, rating=2.7414774894714355), Row(userId=10, rating=2.6314101219177246), Row(userId=28, rating=2.113719940185547), Row(userId=21, rating=2.103858709335327), Row(userId=20, rating=2.0977835655212402), Row(userId=25, rating=2.0574450492858887), Row(userId=1, rating=1.7950433492660522)])

In [25]:
item_recommend.selectExpr("movieId", "explode(recommendations)").show()

+-------+---------------+
|movieId|            col|
+-------+---------------+
|     31| [8, 3.0425339]|
|     31|[14, 2.9733765]|
|     31|  [6, 2.953794]|
|     31| [7, 2.7414775]|
|     31|[10, 2.6314101]|
|     31|  [28, 2.11372]|
|     31|[21, 2.1038587]|
|     31|[20, 2.0977836]|
|     31| [25, 2.057445]|
|     31| [1, 1.7950433]|
|     85|[16, 4.7675653]|
|     85| [8, 4.4026413]|
|     85|[18, 3.7186184]|
|     85| [0, 3.5494723]|
|     85|[14, 3.4386199]|
|     85| [1, 3.0705476]|
|     85|[21, 3.0179274]|
|     85| [6, 3.0144472]|
|     85| [7, 2.8245568]|
|     85|[20, 2.4555118]|
+-------+---------------+
only showing top 20 rows



In [26]:
# COMMAND ----------
from pyspark.ml.evaluation import RegressionEvaluator

In [27]:
evaluator = RegressionEvaluator().setMetricName("rmse").setLabelCol("rating").setPredictionCol("prediction")

In [28]:
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = %f" % rmse)

Root-mean-square error = 1.577370


In [29]:
# COMMAND ----------
from pyspark.mllib.evaluation import RegressionMetrics

In [30]:
predictions.first()

Row(userId=12, movieId=31, rating=4.0, timestamp=1424380312, prediction=-0.19536298513412476)

In [31]:
predictions.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- prediction: float (nullable = false)



In [32]:
regComparisonDF = predictions.select("rating", "prediction")

In [33]:
regComparisonDF.show(2)

+------+-----------+
|rating| prediction|
+------+-----------+
|   4.0|-0.19536299|
|   4.0|  2.8245568|
+------+-----------+
only showing top 2 rows



In [34]:
regComparisonDF.select("rating", "prediction").take(2)

[Row(rating=4.0, prediction=-0.19536298513412476),
 Row(rating=4.0, prediction=2.824556827545166)]

In [35]:
from pyspark.sql.functions import struct, create_map, col

In [36]:
regComparisonDF.selectExpr("struct(rating, prediction) as struct", "*").show(2)

+------------------+------+-----------+
|            struct|rating| prediction|
+------------------+------+-----------+
|[4.0, -0.19536299]|   4.0|-0.19536299|
|  [4.0, 2.8245568]|   4.0|  2.8245568|
+------------------+------+-----------+
only showing top 2 rows



In [37]:
regComparisonDF.selectExpr("struct(rating, prediction) as struct").show(2)

+------------------+
|            struct|
+------------------+
|[4.0, -0.19536299]|
|  [4.0, 2.8245568]|
+------------------+
only showing top 2 rows



In [38]:
regComparison = regComparisonDF.select(create_map(col("rating"), col("prediction")).alias("mapped"))

In [39]:
regComparison.show(3)

+--------------------+
|              mapped|
+--------------------+
|[4.0 -> -0.19536299]|
|  [4.0 -> 2.8245568]|
|  [2.0 -> 1.5255475]|
+--------------------+
only showing top 3 rows



In [40]:
regComparisonDF.rdd.take(2)

[Row(rating=4.0, prediction=-0.19536298513412476),
 Row(rating=4.0, prediction=2.824556827545166)]

In [41]:
print(type(regComparisonDF), type(regComparisonDF.rdd))

(<class 'pyspark.sql.dataframe.DataFrame'>, <class 'pyspark.rdd.RDD'>)


In [42]:
regComparisonrdd = regComparisonDF.rdd

In [43]:
regComparisonrdd.take(3)

[Row(rating=4.0, prediction=-0.19536298513412476),
 Row(rating=4.0, prediction=2.824556827545166),
 Row(rating=2.0, prediction=1.5255475044250488)]

In [44]:
regComparisonrdd.map(lambda x: (x['rating'], x['prediction'])).take(3)

[(4.0, -0.19536298513412476),
 (4.0, 2.824556827545166),
 (2.0, 1.5255475044250488)]

In [45]:
regComparison = regComparisonrdd.map(lambda x: (x['rating'], x['prediction']))

In [46]:
metrics = RegressionMetrics(regComparison)

In [47]:
type(metrics)

pyspark.mllib.evaluation.RegressionMetrics

In [48]:
print(metrics.explainedVariance, 
      metrics.meanAbsoluteError, 
      metrics.meanSquaredError, 
      metrics.r2, 
      metrics.rootMeanSquaredError)

(1.4880244583140434, 1.1701565709065747, 2.4880974895673313, -0.09340558562154255, 1.5773704351126059)


In [49]:
# COMMAND ----------
from pyspark.mllib.evaluation import RankingMetrics, RegressionMetrics
from pyspark.sql.functions import col, expr

In [50]:
perUserActual = predictions\
  .where("rating > 2.5")\
  .groupBy("userId")\
  .agg(expr("collect_set(movieId) as movies"))

In [51]:
perUserActual.show(5)

+------+------------+
|userId|      movies|
+------+------------+
|    27|[33, 27, 55]|
|    26|        [94]|
|    12|    [31, 50]|
|    22|[32, 29, 41]|
|    13|        [83]|
+------+------------+
only showing top 5 rows



In [52]:
# COMMAND ----------
perUserPredictions = predictions\
  .orderBy(col("userId"), expr("prediction DESC"))\
  .groupBy("userId")\
  .agg(expr("collect_list(movieId) as movies"))

In [53]:
perUserPredictions.show(5)

+------+--------------------+
|userId|              movies|
+------+--------------------+
|    28|[60, 56, 45, 90, ...|
|    26|[14, 94, 48, 55, ...|
|    27|    [52, 27, 55, 33]|
|    12| [31, 2, 52, 45, 50]|
|    22|[81, 32, 29, 90, ...|
+------+--------------------+
only showing top 5 rows



In [54]:
# COMMAND ----------
perUserActualvPred = perUserActual.join(perUserPredictions, ["userId"]).rdd.map(lambda row: (row[1], row[2][:15]))

In [55]:
perUserActualvPred.take(2)

[([94], [2, 1, 14, 55, 94, 48, 71]), ([33, 27, 55], [33, 52, 27, 55])]

In [56]:
ranks = RankingMetrics(perUserActualvPred)

In [57]:
# COMMAND ----------
ranks.meanAveragePrecision

0.3187229437229438

In [58]:
ranks.precisionAt(5)
# COMMAND ----------

0.3363636363636364